# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2c000a2e20>
├── 'a' --> tensor([[-0.3705, -1.9046, -0.2274],
│                   [-0.4953, -0.2811, -0.5963]])
└── 'x' --> <FastTreeValue 0x7f2c000a27f0>
    └── 'c' --> tensor([[ 0.3642,  1.7805,  0.0168, -1.6110],
                        [-1.7612, -0.0759,  0.5772,  0.0438],
                        [-0.3412,  0.7690, -0.5320,  0.1889]])

In [4]:
t.a

tensor([[-0.3705, -1.9046, -0.2274],
        [-0.4953, -0.2811, -0.5963]])

In [5]:
%timeit t.a

62.2 ns ± 0.0145 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2c000a2e20>
├── 'a' --> tensor([[-1.7218, -0.4624,  0.8425],
│                   [-0.6429,  0.7356, -1.2068]])
└── 'x' --> <FastTreeValue 0x7f2c000a27f0>
    └── 'c' --> tensor([[ 0.3642,  1.7805,  0.0168, -1.6110],
                        [-1.7612, -0.0759,  0.5772,  0.0438],
                        [-0.3412,  0.7690, -0.5320,  0.1889]])

In [7]:
%timeit t.a = new_value

65.7 ns ± 0.0543 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3705, -1.9046, -0.2274],
               [-0.4953, -0.2811, -0.5963]]),
    x: Batch(
           c: tensor([[ 0.3642,  1.7805,  0.0168, -1.6110],
                      [-1.7612, -0.0759,  0.5772,  0.0438],
                      [-0.3412,  0.7690, -0.5320,  0.1889]]),
       ),
)

In [10]:
b.a

tensor([[-0.3705, -1.9046, -0.2274],
        [-0.4953, -0.2811, -0.5963]])

In [11]:
%timeit b.a

53.7 ns ± 0.0362 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1982,  0.1704,  0.7231],
               [ 1.5033, -0.0825,  2.2077]]),
    x: Batch(
           c: tensor([[ 0.3642,  1.7805,  0.0168, -1.6110],
                      [-1.7612, -0.0759,  0.5772,  0.0438],
                      [-0.3412,  0.7690, -0.5320,  0.1889]]),
       ),
)

In [13]:
%timeit b.a = new_value

477 ns ± 0.0628 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

872 ns ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 23.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 799 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 304 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2bd44bc7f0>
├── 'a' --> tensor([[[-0.3705, -1.9046, -0.2274],
│                    [-0.4953, -0.2811, -0.5963]],
│           
│                   [[-0.3705, -1.9046, -0.2274],
│                    [-0.4953, -0.2811, -0.5963]],
│           
│                   [[-0.3705, -1.9046, -0.2274],
│                    [-0.4953, -0.2811, -0.5963]],
│           
│                   [[-0.3705, -1.9046, -0.2274],
│                    [-0.4953, -0.2811, -0.5963]],
│           
│                   [[-0.3705, -1.9046, -0.2274],
│                    [-0.4953, -0.2811, -0.5963]],
│           
│                   [[-0.3705, -1.9046, -0.2274],
│                    [-0.4953, -0.2811, -0.5963]],
│           
│                   [[-0.3705, -1.9046, -0.2274],
│                    [-0.4953, -0.2811, -0.5963]],
│           
│                   [[-0.3705, -1.9046, -0.2274],
│                    [-0.4953, -0.2811, -0.5963]]])
└── 'x' --> <FastTreeValue 0x7f2c0005c460>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 78 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2bd47dc340>
├── 'a' --> tensor([[-0.3705, -1.9046, -0.2274],
│                   [-0.4953, -0.2811, -0.5963],
│                   [-0.3705, -1.9046, -0.2274],
│                   [-0.4953, -0.2811, -0.5963],
│                   [-0.3705, -1.9046, -0.2274],
│                   [-0.4953, -0.2811, -0.5963],
│                   [-0.3705, -1.9046, -0.2274],
│                   [-0.4953, -0.2811, -0.5963],
│                   [-0.3705, -1.9046, -0.2274],
│                   [-0.4953, -0.2811, -0.5963],
│                   [-0.3705, -1.9046, -0.2274],
│                   [-0.4953, -0.2811, -0.5963],
│                   [-0.3705, -1.9046, -0.2274],
│                   [-0.4953, -0.2811, -0.5963],
│                   [-0.3705, -1.9046, -0.2274],
│                   [-0.4953, -0.2811, -0.5963]])
└── 'x' --> <FastTreeValue 0x7f2c00095340>
    └── 'c' --> tensor([[ 0.3642,  1.7805,  0.0168, -1.6110],
                        [-1.7612, -0.0759,  0.5772,  0.0438],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 25.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.3 µs ± 412 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.3642,  1.7805,  0.0168, -1.6110],
                       [-1.7612, -0.0759,  0.5772,  0.0438],
                       [-0.3412,  0.7690, -0.5320,  0.1889]],
              
                      [[ 0.3642,  1.7805,  0.0168, -1.6110],
                       [-1.7612, -0.0759,  0.5772,  0.0438],
                       [-0.3412,  0.7690, -0.5320,  0.1889]],
              
                      [[ 0.3642,  1.7805,  0.0168, -1.6110],
                       [-1.7612, -0.0759,  0.5772,  0.0438],
                       [-0.3412,  0.7690, -0.5320,  0.1889]],
              
                      [[ 0.3642,  1.7805,  0.0168, -1.6110],
                       [-1.7612, -0.0759,  0.5772,  0.0438],
                       [-0.3412,  0.7690, -0.5320,  0.1889]],
              
                      [[ 0.3642,  1.7805,  0.0168, -1.6110],
                       [-1.7612, -0.0759,  0.5772,  0.0438],
                       [-0.3412,  0.7690, -0.5320,  0.1889]],

In [26]:
%timeit Batch.stack(batches)

79.2 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.3642,  1.7805,  0.0168, -1.6110],
                      [-1.7612, -0.0759,  0.5772,  0.0438],
                      [-0.3412,  0.7690, -0.5320,  0.1889],
                      [ 0.3642,  1.7805,  0.0168, -1.6110],
                      [-1.7612, -0.0759,  0.5772,  0.0438],
                      [-0.3412,  0.7690, -0.5320,  0.1889],
                      [ 0.3642,  1.7805,  0.0168, -1.6110],
                      [-1.7612, -0.0759,  0.5772,  0.0438],
                      [-0.3412,  0.7690, -0.5320,  0.1889],
                      [ 0.3642,  1.7805,  0.0168, -1.6110],
                      [-1.7612, -0.0759,  0.5772,  0.0438],
                      [-0.3412,  0.7690, -0.5320,  0.1889],
                      [ 0.3642,  1.7805,  0.0168, -1.6110],
                      [-1.7612, -0.0759,  0.5772,  0.0438],
                      [-0.3412,  0.7690, -0.5320,  0.1889],
                      [ 0.3642,  1.7805,  0.0168, -1.6110],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 8.29 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
